<a href="https://colab.research.google.com/github/cotozelo/Precisa_Operacional/blob/main/AHP_Sigma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AHP Sigma

In [ ]:
import pandas as pd

### Atributos por Produto

In [ ]:
df_carros = pd.DataFrame({
'Nome':          ['Fiesta', 'Uno', 'Gol', 'March', 'Sandero'  ],
'Preco [R$]':    [31000   , 30000, 35000,  32000,   31000],
'Potencia [cv]': [107     ,    88,    75,    111,   107],
'Motor [L]':     [1.6     ,   1.4,   1.0,    1.6,   1.6],
'Torque [kgfm]': [15.3    ,  12.3,  10.6,   15.1,  15.3]})

df_carros


,Nome,Preco [R$],Potencia [cv],Motor [L],Torque [kgfm]
0,Fiesta,31000,107,1.6,15.3
1,Uno,30000,88,1.4,12.3
2,Gol,35000,75,1.0,10.6
3,March,32000,111,1.6,15.1
4,Sandero,31000,107,1.6,15.3


### Identifica influencia positiva e negativa

* Influencia negativa: ```influencia_atributo = 1/valor_atributo```

* Influencia positiva: ```influencia_atributo = valor_atribulto```

In [ ]:
# Influencia Negativa
inf_preco = 1/df_carros['Preco [R$]']

# Influencia Positiva
inf_potencia = df_carros['Potencia [cv]']
inf_motor    = df_carros['Motor [L]']
inf_toque    = df_carros['Torque [kgfm]']

### Aplicando normalização baseada na influencia

```atributo_normalizado = influencia_atributo/Sum(influencia_atributo)```

In [ ]:
df_car_norm = df_carros.copy()

df_car_norm['Preco [R$]']    = inf_preco/inf_preco.sum()
df_car_norm['Potencia [cv]'] = inf_potencia/inf_potencia.sum()
df_car_norm['Motor [L]']     = inf_motor/inf_motor.sum()
df_car_norm['Torque [kgfm]'] = inf_toque/inf_toque.sum()
df_car_norm

,Nome,Preco [R$],Potencia [cv],Motor [L],Torque [kgfm]
0,Fiesta,0.204591,0.219262,0.222222,0.223032
1,Uno,0.211411,0.180328,0.194444,0.179300
2,Gol,0.181209,0.153689,0.138889,0.154519
3,March,0.198198,0.227459,0.222222,0.220117
4,Sandero,0.204591,0.219262,0.222222,0.223032


### Desvio Padrão por Atributo

```desvio_atributo```

In [ ]:
sigma = df_car_norm.describe().loc['std', :].to_dict()
sigma

{'Preco [R$]': 0.011496775635213294,
 'Potencia [cv]': 0.031732534973851395,
 'Motor [L]': 0.03621779114001473,
 'Torque [kgfm]': 0.031474730406029}

### Média por Atributo

```media_atributo```

In [ ]:
media = df_car_norm.describe().loc['mean', :].to_dict()
media

{'Preco [R$]': 0.2,
 'Potencia [cv]': 0.2,
 'Motor [L]': 0.20000000000000004,
 'Torque [kgfm]': 0.19999999999999998}

### Nomalizando Desvio Padrão

```desvio_atributo_normalizado = desvio_atributo / sum(desvio_atributo)```

In [ ]:
#@title Desvio Padrão Normalizado ()
sigma_normal = {}
sigma_soma = sum(list(sigma.values()))
for kk in sigma:
    sigma_normal[kk] = sigma[kk]/sigma_soma
sigma_normal

{'Preco [R$]': 0.103647545409606,
 'Potencia [cv]': 0.28608015534289005,
 'Motor [L]': 0.3265163443150605,
 'Torque [kgfm]': 0.28375595493244343}

### Peso por Atributo

```peso = atributo_normalizado * desvio_atributo_normalizado ```

In [ ]:
df_car_import_sigma = df_car_norm.copy()
df_car_import_sigma['Preco [R$]'] = df_car_norm['Preco [R$]'] * sigma_normal['Preco [R$]']
df_car_import_sigma['Potencia [cv]'] = df_car_norm['Potencia [cv]'] * sigma_normal['Potencia [cv]']
df_car_import_sigma['Motor [L]'] = df_car_norm['Motor [L]'] * sigma_normal['Motor [L]']
df_car_import_sigma['Torque [kgfm]'] = df_car_norm['Torque [kgfm]'] * sigma_normal['Torque [kgfm]']
df_car_import_sigma

,Nome,Preco [R$],Potencia [cv],Motor [L],Torque [kgfm]
0,Fiesta,0.021205,0.062727,0.072559,0.063287
1,Uno,0.021912,0.051588,0.063489,0.050878
2,Gol,0.018782,0.043967,0.045349,0.043846
3,March,0.020543,0.065072,0.072559,0.062459
4,Sandero,0.021205,0.062727,0.072559,0.063287


### Score

```Score = Sum(peso)```

In [ ]:
#@title Somando a linha e encontrando o score e o melhor custo benefício
df_car_custo_beneficio_sigma = pd.DataFrame({
    'Modelo': ['Fiesta', 'Uno', 'Gol', 'March', 'Sandero'],
    'Score':  [df_car_import_sigma[df_car_import_sigma['Nome'] == 'Fiesta'].T[0].iloc[1:].sum(),
               df_car_import_sigma[df_car_import_sigma['Nome'] == 'Uno'].T[1].iloc[1:].sum(),
               df_car_import_sigma[df_car_import_sigma['Nome'] == 'Gol'].T[2].iloc[1:].sum(),
               df_car_import_sigma[df_car_import_sigma['Nome'] == 'March'].T[3].iloc[1:].sum(),
               df_car_import_sigma[df_car_import_sigma['Nome'] == 'Sandero'].T[4].iloc[1:].sum()]})
df_car_custo_beneficio_sigma.sort_values('Score', inplace=True, ascending=False)
df_car_custo_beneficio_sigma

,Modelo,Score
3,March,0.220633
0,Fiesta,0.219778
4,Sandero,0.219778
1,Uno,0.187867
2,Gol,0.151944
